In [55]:
from typing import Sequence, FrozenSet, Any
from pydantic import confrozenset
from pydantic.dataclasses import dataclass
from pydantic import BaseModel
from apischema import serialize, deserialize
from frozendict import frozendict
import json

In [56]:
class FrozenSetEncoder(json.JSONEncoder):
    def default(self, o: Any) -> Any:
        if isinstance(o, FrozenSet):
            return list(o)
        return json.JSONEncoder.default(self, o)

In [58]:
class Model(BaseModel):
    def dict(self, *args, **kwargs):
        d = super().dict(*args, **kwargs)
        return frozendict(**d)
        
    class Config:
        frozen = True

In [59]:
# @dataclass(frozen=True, config=Config)
class Bar(Model, frozen=True):
    x: int

In [60]:
# @dataclass(frozen=True, config=Config)
class Foo(Model, frozen=True):
    bars: confrozenset(Bar)

In [61]:
foo1 = Foo(bars=[Bar(x=1), Bar(x=2)])
foo2 = Foo(bars=[Bar(x=2), Bar(x=1), Bar(x=1)])

In [62]:
foo1.dict()

frozendict.frozendict({'bars': frozenset({frozendict.frozendict({'x': 1}), frozendict.frozendict({'x': 2})})})

In [63]:
foo1 == foo2

True

In [64]:
from fastapi.encoders import jsonable_encoder
jsonable_encoder(foo1)

{'bars': [{'x': 1}, {'x': 2}]}